In [1]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [2]:

import cv2
import math

import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import size
from matplotlib import colors
from asgiref.sync import sync_to_async #in order to load variables to communicate between sync and async

import optimal_path as op

from robot import Robot
from Map import Map
from Local_navigation import* 
from Mouvement import*
from vision import*
from optimal_path import* 


map_lenght = 1
nb_of_square_by_side = 50
current = 1

global move 
move = False #bool which indicate how our robot is moving (1: avoid an object VS 0: following the optimal path)
kalman_bool = False
old_distance = 0
old_angle = 0
ERR_POS = 3
#pourcentage_reduc = 1

In [3]:


#VideoCap = vision_initialization()
VideoCap=cv2.VideoCapture(0)


In [4]:
@tdmclient.notebook.sync_var
def motors(l_speed=500, r_speed=500, verbose=False):
    """
    Sets the motor speeds of the Thymio 
    param l_speed: left motor speed
    param r_speed: right motor speed
    param verbose: whether to print status messages or not
    """
    global motor_left_target, motor_right_target
    # Printing the speeds if requested
    if verbose:
        print("\t\t Setting speed : ", l_speed, r_speed)
    motor_left_target = l_speed
    motor_right_target = r_speed

In [5]:
@tdmclient.notebook.sync_var
def proxi():
    """
    Returns the proximity values of the Thymio 
    """
    global prox_horizontal
    return prox_horizontal


In [6]:
#############################
#Test MASK  A LANCER QUE POUR FAIRE DES TESTS SUR LE MASK
#############################

while True:
    ret, frame = VideoCap.read()
    mask, b = mask_map_init(frame)
    
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        break
    cv2.imshow("mask",mask)
    cv2.imshow("mask B",b)
    #pos_robot, angle = update(frame, 0.5)


In [7]:
George = Robot()
Lausanne = Map(map_lenght, nb_of_square_by_side)

In [8]:
VideoCap=cv2.VideoCapture(0)
while(True):
    ret, frame=VideoCap.read()
    
    
    bl_points, bl_mask, bl_contours = detect_inrange(frame, 1000, blue)
    gr_points, gr_mask, gr_contours = detect_inrange(frame, 1000, green)
    
    red_points, red_mask, red_contours = detect_inrange(frame, 400, red) 
    
    if(len(red_points)>1):
        cv2.arrowedLine(frame,
                    (int(red_points[0][0]), int(red_points[0][1])), (int(red_points[1][0]), int(red_points[1][1])),
                    color=(0, 255, 0),
                    thickness=3,
                    tipLength=0.2)
    
    cv2.imshow('image bl', bl_mask)
    cv2.imshow('image gr', gr_mask)
    cv2.imshow('image red', red_mask)
 
          
    if cv2.waitKey(1)&0xFF==ord('q'):
        print('le bouton quitter')        
        vision_end(VideoCap)
        break 



le bouton quitter


In [8]:
#np.set_printoptions(threshold=sys.maxsize)
####################################
## INIT
####################################
nb_frame_to_take = 10
# Prend une première image
for i in range (nb_frame_to_take): 
    ret, frame = VideoCap.read()
    
print("taille frame", frame.shape)
cv2.imwrite("debut.jpg", frame)

print("MAP INITIALISATION")
# Set the pourcentage value between the nb of pixel and nb of square by side
Lausanne.set_map_lenght(frame)
#print("Pourcentage",Lausanne.get_pourcentage())
# Set the robot goal, position and angle
goal = init_goal(frame, Lausanne.get_pourcentage())
#goal = (50, 40)

# Set pour relancer direct
pos_robot = (-1,-1)
George.set_pos(pos_robot) 
#print("GOAL", goal)

print("first while")
#pos_robot, angle = update(frame, Lausanne.get_pourcentage())  # fonction qui retourne la position angle etc....
 
      #Utiliser au cas où il ne trouve pas la bonne position du premier coup
while (pos_robot[0] <= 0) & (pos_robot[1] <= 0):

    #ret, frame=VideoCap.read()
    #display(frame,1,0,0)
    print("searching for the strart pos")
    ret, frame=VideoCap.read()
    #vs.display (frame, 0, 1, 1, 0, 5, Lausanne.get_pourcentage())
    pos_robot, angle, bool_measure = update(frame, Lausanne.get_pourcentage())  # fonction qui retourne la position angle etc....
    #print("Waiting robot position") 

George.set_angle(angle)
George.set_goal(goal)
George.set_start_pos(pos_robot)
George.set_pos(pos_robot)
print ("goal", goal)
print ("start position", pos_robot)

# SET MAP
Lausanne.set_map_lenght(frame)
#print("Lausanna lenght", Lausanne.get_lenght())

# MASK 
mask_bleu, mask_green = mask_map_init(frame)
#cv2.imshow("mask", mask)
cv2.imwrite("masque1.jpg", mask_bleu)
cv2.imwrite("masque2.jpg", mask_green)

# Fin de l'init
Lausanne.init_grid(mask_bleu,mask_green)
grille = Lausanne.get_map()
#cv2.imshow("grid", grille)
cv2.imwrite("grille.jpg", grille)

# OPTIMAL PATH
print("PATH COMPUTATION")
path, visitedNodes = op.path_computation(George.get_start() , George.get_goal() , Lausanne.get_lenght(), grille)

George.set_path(path) 
George.set_visit_nodes(visitedNodes) 

# Display the optimal path
op.display_map(Lausanne.get_lenght(),  grille,  George.get_visit_nodes(), George.get_path(), George.get_start(), George.get_goal())
#print("PATH", George.get_path())

display (frame, 0, 1, 1, 1, George.get_path(), Lausanne.get_pourcentage())
#print("PATH22222", George.get_path())
cv2.imwrite("TOTAL.jpg", frame)

#vision_end(VideoCap)



taille frame (480, 640, 3)
MAP INITIALISATION
first while
searching for the strart pos
goal (18, 34)
start position (43, 40)
PATH COMPUTATION


Exception: Start node is not traversable

In [8]:
import time

In [9]:

####################################
## while
####################################

George.set_current(0)
current= 0
bool_measure = False
#print("PATH", George.get_path())
old_distance = 0
old_angle = 0
soon_arrived = False
thresh_indentation = 15

KF=KalmanFilter(0.1, [0.0, 0])
x_old = 0
y_old = 0
position_robot =np.array([0,0])

print('START MOUVEMENT')
while True:#(George.get_pos() != George.get_goal()):
    start = time.time()
    
    etat = KF.predict().astype(np.float64)
    #print('etat est de la forme',etat)
    #print(etat[0,0])
    
       
    position_robot[0] = etat[0][0]
    position_robot[1] = etat[1][0]
    
    # Read the camera
    ret, frame=VideoCap.read()
    
    # Recupere la position et angle --> XAV
    pos_camera,angle_robot,bool_measure = update(frame, Lausanne.get_pourcentage())
    
    if bool_measure:
        etat
        speed = ( 100 + 100)/ 2
        speed = speed / 45.045
        print(' la vitesse moyen est', speed)

        speed_x = speed*np.cos(angle_robot)
        speed_y = speed*np.sin(angle_robot)

        if (len(pos_robot)>0):
            array = np.array([ 0,0,0,0.0])
            array[0] = float(pos_camera[0])
            array[1] = float(pos_camera[1])
            array[2] = float(speed_x)
            array[3] = float(speed_y)

            KF.update(np.expand_dims(array, axis=-1))
            
    print(' bool detect est',  bool_measure)
    print('la position du robot est', position_robot)
    print(' les points du kF sont', etat)
    
    
    George.set_pos(position_robot)
    George.set_angle(angle_robot)
    
    if(current < George.get_path().shape[1]-1):
        current = path_update(position_robot, ERR_POS, np.transpose(George.get_path())[current], George.get_current())
        George.set_current(current)
    else:
        print("ARRIVED")
        motors(0,0)
        break
    
    #print("pos", pos_robot)
    print("taille", George.get_path().shape[1])
    print("current", current)
    pos_goal =  np.transpose(George.get_path())[current]
    
    if current > (George.get_path().shape[1] - thresh_indentation) :
        soon_arrived = True
        print('soon arrived')
    
    proximity = proxi()
    print ('proximity', proximity)
    move = check_cars(prox_horizonta=proximity)
    print('move check cars', move)
    """
        #LOCAL NAVIGATION
    if move and not (soon_arrived):
        #Thymio is avoiding obstacles
        print("OBSTACLLLEEEE")
        speed_l, speed_r, move = avoid_obstacle(prox_horizonta=proximity) 
    else:
        #Thymio is following the optimal path
        print("Avancee")
        speed_l, speed_r, old_distance, old_goal  = move_to_position(pos_robot, angle_robot, pos_goal, old_distance, old_angle)
        #Thymio is checking if there's an obstacle in front of it
        move = check_cars(prox_horizonta=proximity)
        print (speed_l, speed_r)
       
    """
    speed_l, speed_r, old_distance, old_goal  = move_to_position(position_robot, angle_robot, pos_goal, old_distance, old_angle)
    motors(speed_l, speed_r) 
    
    
    display (frame, 0, 1, 1, 1, path, Lausanne.get_pourcentage())
    
    end = time.time()

    print('le temps d execution est de',end - start)
    
    # Quit when we press q
    keyVal = cv2.waitKey(1) & 0xFF
    if keyVal == ord('q'):
        motors(0,0)
        break
     
    sleep(0.1)

print("The End")
    

START MOUVEMENT
 la vitesse moyen est 2.22000222000222
update 2 E [[23.]
 [28.]
 [-2.]
 [ 1.]]
 bool detect est True
la position du robot est [0 0]
 les points du kF sont [[0.]
 [0.]
 [0.]
 [0.]]
PATH [35 42]
X 35
Y 42
taille 18
current 0
proximity [0, 0, 0, 0, 0, 0, 0]
move check cars False
movement
[0 0]
old dist, 0 old angle 0
debut 0 0 2.89234051220458
dist_debut 54.67174773134658
-2.016282461606387
new dist, 54.67174773134658 new angle -2.016282461606387
speed_original, 403 -403
speed, 200 -403
le temps d execution est de 0.016003131866455078
 la vitesse moyen est 2.22000222000222
update 2 E [[30.]
 [37.]
 [-2.]
 [ 1.]]
 bool detect est True
la position du robot est [22 28]
 les points du kF sont [[22.8]
 [28.1]
 [-2. ]
 [ 1. ]]
PATH [35 42]
X 13
Y 14
taille 18
current 0
proximity [0, 0, 0, 0, 0, 0, 0]
move check cars False
movement
[22 28]
old dist, 54.67174773134658 old angle 0
debut 22 28 2.8879185574511506
dist_debut 19.1049731745428
-2.065500278179772
new dist, 19.10497317454

 la vitesse moyen est 2.22000222000222
update 2 E [[36.]
 [41.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [35 41]
 les points du kF sont [[35.8]
 [41. ]
 [-2. ]
 [ 0. ]]
PATH [32 39]
X 3
Y 2
taille 18
current 3
proximity [0, 0, 0, 0, 0, 0, 0]
move check cars False
movement
[35 41]
old dist, 3.605551275463989 old angle 0
debut 35 41 -2.905347392871356
dist_debut 3.605551275463989
6.634942650008716
new dist, 3.605551275463989 new angle 0.35175734282913007
speed_original, 26 167
speed, 26 167
le temps d execution est de 0.032007455825805664
 la vitesse moyen est 2.22000222000222
update 2 E [[36.]
 [41.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [35 40]
 les points du kF sont [[35.8]
 [40.9]
 [-2. ]
 [-1. ]]
PATH [32 39]
X 3
Y 1
taille 18
current 3
proximity [0, 0, 0, 0, 0, 0, 0]
move check cars False
movement
[35 40]
old dist, 3.605551275463989 old angle 0
debut 35 40 -2.764343593947434
dist_debut 3.1622776601683795
6.227686801933869
new dist, 3.162

 la vitesse moyen est 2.22000222000222
update 2 E [[33.]
 [39.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [32 38]
 les points du kF sont [[32.8]
 [38.9]
 [-2. ]
 [-1. ]]
PATH [29 36]
X 3
Y 2
taille 18
current 6
soon arrived
proximity [0, 0, 0, 795, 0, 0, 0]
move check cars False
movement
[32 38]
old dist, 3.605551275463989 old angle 0
debut 32 38 -2.5088438185876103
dist_debut 3.605551275463989
6.238439075724971
new dist, 3.605551275463989 new angle -0.044746231454615426
speed_original, 106 88
speed, 106 88
le temps d execution est de 0.031012535095214844
 la vitesse moyen est 2.22000222000222
update 2 E [[32.]
 [39.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [32 38]
 les points du kF sont [[32.8]
 [38.9]
 [-2. ]
 [-1. ]]
PATH [29 36]
X 3
Y 2
taille 18
current 6
soon arrived
proximity [0, 0, 0, 820, 0, 0, 0]
move check cars False
movement
[32 38]
old dist, 3.605551275463989 old angle 0
debut 32 38 -2.49454546036947
dist_debut 3.605551275463989
6.

 la vitesse moyen est 2.22000222000222
update 2 E [[30.]
 [37.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [29 36]
 les points du kF sont [[29.8]
 [36.9]
 [-2. ]
 [-1. ]]
PATH [26 35]
X 3
Y 1
taille 18
current 9
soon arrived
proximity [0, 0, 2084, 3748, 0, 0, 0]
move check cars True
movement
[29 36]
old dist, 3.1622776601683795 old angle 0
debut 29 36 -2.723368324010564
dist_debut 3.1622776601683795
6.186711531996999
new dist, 3.1622776601683795 new angle -0.09647377518258704
speed_original, 115 77
speed, 115 77
le temps d execution est de 0.0340580940246582
 la vitesse moyen est 2.22000222000222
update 2 E [[30.]
 [37.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [29 36]
 les points du kF sont [[29.8]
 [36.9]
 [-2. ]
 [-1. ]]
PATH [26 35]
X 3
Y 1
taille 18
current 9
soon arrived
proximity [0, 0, 2669, 4122, 0, 0, 0]
move check cars True
movement
[29 36]
old dist, 3.1622776601683795 old angle 0
debut 29 36 -2.7455112120254856
dist_debut 3.1622776601

 la vitesse moyen est 2.22000222000222
update 2 E [[28.]
 [35.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [27 35]
 les points du kF sont [[27.8]
 [35.9]
 [-2. ]
 [-1. ]]
PATH [24 34]
X 3
Y 1
taille 18
current 11
soon arrived
proximity [0, 2756, 4653, 4799, 0, 0, 0]
move check cars True
movement
[27 35]
old dist, 3.1622776601683795 old angle 0
debut 27 35 -2.9094888155146377
dist_debut 3.1622776601683795
6.372832023501073
new dist, 3.1622776601683795 new angle 0.08964671632148669
speed_original, 78 114
speed, 78 114
le temps d execution est de 0.030011892318725586
 la vitesse moyen est 2.22000222000222
update 2 E [[28.]
 [35.]
 [-2.]
 [-1.]]
 bool detect est True
la position du robot est [27 34]
 les points du kF sont [[27.8]
 [34.9]
 [-2. ]
 [-1. ]]
PATH [24 34]
X 3
Y 0
taille 18
current 11
soon arrived
proximity [748, 2969, 4573, 4611, 0, 0, 0]
move check cars True
movement
[27 34]
old dist, 3.1622776601683795 old angle 0
debut 27 34 -2.905347392871356
dist_debut 3

 la vitesse moyen est 2.22000222000222
update 2 E [[28.]
 [35.]
 [-2.]
 [-0.]]
 bool detect est True
la position du robot est [27 35]
 les points du kF sont [[27.8]
 [35. ]
 [-2. ]
 [ 0. ]]
PATH [24 34]
X 3
Y 1
taille 18
current 11
soon arrived
proximity [2316, 3945, 4575, 4687, 0, 0, 0]
move check cars True
movement
[27 35]
old dist, 3.1622776601683795 old angle 0
debut 27 35 -3.03671571485956
dist_debut 3.1622776601683795
6.500058922845995
new dist, 3.1622776601683795 new angle 0.21687361566640906
speed_original, 52 139
speed, 52 139
le temps d execution est de 0.030988693237304688
 la vitesse moyen est 2.22000222000222
update 2 E [[28.]
 [35.]
 [-2.]
 [-0.]]
 bool detect est True
la position du robot est [27 35]
 les points du kF sont [[27.8]
 [35. ]
 [-2. ]
 [ 0. ]]
PATH [24 34]
X 3
Y 1
taille 18
current 11
soon arrived
proximity [2182, 3943, 4715, 4748, 0, 0, 0]
move check cars True
movement
[27 35]
old dist, 3.1622776601683795 old angle 0
debut 27 35 -3.054097312787402
dist_debut

In [9]:
vision_end(VideoCap)

In [10]:
motors(0,0)